In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

pd.set_option("display.max_rows", 2000)

In [17]:

# Page 33 of 33
# or
# 500 per page
# so thinking do an if statement on the last value of that string, if its an int use it as the number of pages, if not just take that page. 


In [18]:
def find_bottles(bottles):

    bottle_ids = bottles.find_all("h6")                                      
    bottle_names = bottles.find_all("h4")[3:]
    all_bottles_prices = bottles.find_all(class_= ["sold", "bndone"])

    put_together = list(zip(bottle_ids, bottle_names, all_bottles_prices)) # putting all the info into a list

    df = pd.DataFrame(put_together, columns=["Lot number", "Name", "Price"])
    for i in df.columns:
        df[i] = df[i].apply(lambda x: BeautifulSoup(str(x), "html.parser").get_text())
    df = df.loc[~df["Price"].str.contains("This lot is no longer available")]  # Getting just sold bottles

    return df

#bottles = soup
#df = find_bottles(bottles)
#df.head(5)


In [19]:
link2 = "https://www.scotchwhiskyauctions.com/auctions/" 
webpage_response2 = requests.get(link2)

webpage2 = webpage_response2.content
soup2 = BeautifulSoup(webpage2, "lxml")

auction_links = [i["href"] for i in soup2.find_all("a", class_="auction", href=True)] # Getting the auction links
auction_links.reverse()
print(auction_links[0])

auctions/14-the-inaugural-auction/


In [20]:
#link = "https://www.scotchwhiskyauctions.com/auctions/181-the-137th-auction/?page=1"
#webpage_response =requests.get(link)
#
#webpage = webpage_response.content
#soup = BeautifulSoup(webpage, "lxml")

df = pd.DataFrame(columns=["Lot number", "Name", "Price"])
count = 0
auction_count = 0
if __name__ == "__main__":
    # Iterating through all the auctions
    for auction in auction_links:
        print("-----") # Make the terminal a bit easier to read
        auction_count += 1
        print(f"Auction number: {auction_count}")
        
        webpage_response = requests.get("https://www.scotchwhiskyauctions.com/" + auction + "?q=macallan&search=c") #   Getting a auction page

        webpage = webpage_response.content
        soup = BeautifulSoup(webpage, "lxml")

        # Getting the number of pages for each auction
        number_pages_tag = soup.find_all("option")
        total_auction_pages_list = [i.text for i in number_pages_tag][-3].split() # Getting the pages string, and putting in a list to get the last value
        
        number_of_pages = total_auction_pages_list[-1] # Getting the pages last value (being the total number of pages on that auction, or "page" if there was only one page)
        
        
        try: 
            for page in range(1, int(number_of_pages)): # Need the number of pages for each auction
                link = "https://www.scotchwhiskyauctions.com/"
                link = link + auction + "?q=macallan&search=c" + "&page=" + str(page)
                webpage_response = requests.get(link)


                webpage = webpage_response.content
                soup = BeautifulSoup(webpage, "lxml")

                current_page = find_bottles(soup)
                df = pd.concat([df, current_page], ignore_index=True)
                #print(current_page.values)

                time.sleep(1) 
                count += 1
                print(f"Scraped: {count} pages")
        except:
            link = "https://www.scotchwhiskyauctions.com/"

            # In this one we dont have the page number
            link = link + auction + "?q=macallan&search=c"    
            webpage_response = requests.get(link)
            webpage = webpage_response.content
            soup = BeautifulSoup(webpage, "lxml")

            current_page = find_bottles(soup)
            df = pd.concat([df, current_page], ignore_index=True)
            #print(current_page.values)

            time.sleep(1) 
            count += 1
            print(f"Scraped: {count} pages")
            


df["Price"] = df["Price"].str.replace(",", "") # Removing the commas in the price column
df["Price"] = df["Price"].str.extract("Sold for £(\d+)") #Capturing just the prices. 
    
df["Price"] = df["Price"].astype(int) # Putting the prices as an int
#df.drop(columns=["Unnamed: 0"], inplace=True) # Dropping the other index as a column

#df.to_csv("Macallan 75th.csv")


-----
Auction number: 1
Scraped: 1 pages
-----
Auction number: 2
Scraped: 2 pages
-----
Auction number: 3
Scraped: 3 pages
-----
Auction number: 4
Scraped: 4 pages
-----
Auction number: 5
Scraped: 5 pages
-----
Auction number: 6
Scraped: 6 pages
-----
Auction number: 7
Scraped: 7 pages
-----
Auction number: 8
Scraped: 8 pages
-----
Auction number: 9
Scraped: 9 pages
-----
Auction number: 10
Scraped: 10 pages
-----
Auction number: 11
Scraped: 11 pages
-----
Auction number: 12
Scraped: 12 pages
-----
Auction number: 13
Scraped: 13 pages
-----
Auction number: 14
Scraped: 14 pages
-----
Auction number: 15
Scraped: 15 pages
Scraped: 16 pages
-----
Auction number: 16
Scraped: 17 pages
-----
Auction number: 17
Scraped: 18 pages
-----
Auction number: 18
Scraped: 19 pages
-----
Auction number: 19
Scraped: 20 pages
-----
Auction number: 20
Scraped: 21 pages
-----
Auction number: 21
Scraped: 22 pages
-----
Auction number: 22
Scraped: 23 pages
Scraped: 24 pages
-----
Auction number: 23
Scraped: 25

In [ ]:
# We need to get all the links so we can iterate through
# We then need iterate through each of the links and each of the pages using an imbedded for loop
# We also need to figure out a way to update the page part of the for loop so that we automatically change the pages